# 🛒 간단 쇼핑몰 · 리뷰 분석 서비스 + LLM

이 노트북은 **SQLite를 데이터베이스로 사용**하고,
**LLM(OpenAI)을 이용해 자연어 질의를 SQL로 변환**하여
간단한 쇼핑몰/리뷰 분석 서비스를 구현하는 예제입니다.

비전공자 학부생도 이해할 수 있도록, 각 단계마다 **개념 설명과 주석**을 충분히 넣었습니다.

구성:
1. SQLite DB 생성 및 스키마 설계
2. 샘플 데이터 삽입
3. 기본 SQL 질의 연습
4. OpenAI LLM 연동 (자연어 → SQL 변환)
5. 자연어로 쇼핑/리뷰 데이터 질의하기
6. (선택) 매출·리뷰 요약 리포트 자동 생성


## 0. 환경 준비

- 이 노트북은 **Google Colab**에서 실행하는 것을 기준으로 작성했습니다.
- Colab에서 실행할 때는 `openai` 라이브러리를 먼저 설치해야 합니다.


In [1]:
# ⬇️ Colab에서 한 번만 실행하면 됩니다.
# 이 셀은 OpenAI Python 클라이언트를 설치합니다.
# 로컬 환경에서는 이미 설치돼 있을 수도 있습니다.

!pip install --quiet openai pandas


In [2]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import json
import textwrap
import os
from openai import OpenAI

print("✅ 라이브러리 임포트 완료")


✅ 라이브러리 임포트 완료


## 1. SQLite 데이터베이스 생성 및 스키마 설계

### 도메인: 간단 쇼핑몰 · 리뷰 서비스

우리가 만들 데이터베이스에는 다음 5개의 테이블이 있습니다.

1. `users` : 사용자 정보
2. `products` : 상품 정보
3. `orders` : 주문(주문 단위, 한 번 결제)
4. `order_items` : 주문 안에 포함된 개별 상품들
5. `reviews` : 사용자들이 남긴 상품 리뷰

모든 데이터는 **하나의 SQLite DB 파일(메모리 또는 파일)** 안에 저장됩니다.


In [3]:
# 메모리 DB를 사용할 수도 있고, 파일로 저장할 수도 있습니다.
# 편의를 위해 여기서는 파일로 저장해 보겠습니다.

DB_PATH = "shop_llm_db.sqlite"
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

print(f"✅ SQLite DB 연결 완료: {DB_PATH}")


✅ SQLite DB 연결 완료: shop_llm_db.sqlite


In [4]:
# 기존에 같은 이름의 테이블이 있다면 삭제해서 항상 같은 상태에서 시작하도록 합니다.

tables = ["order_items", "reviews", "orders", "products", "users"]
for t in tables:
    cur.execute(f"DROP TABLE IF EXISTS {t}")

conn.commit()
print("🧹 기존 테이블 정리 완료")


🧹 기존 테이블 정리 완료


In [5]:
# 1) users 테이블: 쇼핑몰 사용자
cur.execute(
    """
    CREATE TABLE users (
        id          INTEGER PRIMARY KEY AUTOINCREMENT,
        name        TEXT NOT NULL,
        email       TEXT NOT NULL UNIQUE,
        join_date   TEXT NOT NULL
    );
    """
)

# 2) products 테이블: 상품
cur.execute(
    """
    CREATE TABLE products (
        id          INTEGER PRIMARY KEY AUTOINCREMENT,
        name        TEXT NOT NULL,
        category    TEXT NOT NULL,
        price       REAL NOT NULL
    );
    """
)

# 3) orders 테이블: 주문(결제 단위)
cur.execute(
    """
    CREATE TABLE orders (
        id            INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id       INTEGER NOT NULL,
        order_date    TEXT NOT NULL,
        total_amount  REAL NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id)
    );
    """
)

# 4) order_items 테이블: 주문 안에 포함된 개별 상품
cur.execute(
    """
    CREATE TABLE order_items (
        id          INTEGER PRIMARY KEY AUTOINCREMENT,
        order_id    INTEGER NOT NULL,
        product_id  INTEGER NOT NULL,
        quantity    INTEGER NOT NULL,
        price       REAL NOT NULL,
        FOREIGN KEY (order_id) REFERENCES orders(id),
        FOREIGN KEY (product_id) REFERENCES products(id)
    );
    """
)

# 5) reviews 테이블: 상품 리뷰
cur.execute(
    """
    CREATE TABLE reviews (
        id          INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id     INTEGER NOT NULL,
        product_id  INTEGER NOT NULL,
        rating      INTEGER NOT NULL, -- 1~5 점수
        comment     TEXT,
        created_at  TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id),
        FOREIGN KEY (product_id) REFERENCES products(id)
    );
    """
)

conn.commit()
print("✅ 테이블 생성 완료")


✅ 테이블 생성 완료


## 2. 샘플 데이터 삽입

실제 서비스처럼 테스트하려면 **적당한 양의 샘플 데이터**가 필요합니다.

- 사용자: 5명 정도
- 상품: 여러 카테고리(전자제품, 도서, 생활용품 등)로 8~10개
- 주문: 최근 60일 안에 여러 건
- 주문 아이템: 주문당 1~3개 정도
- 리뷰: 인기 있는 상품에 리뷰 여러 개


In [6]:
# 2-1) users 데이터 삽입
users_data = [
    ("김철수", "chulsoo@example.com", "2025-01-10"),
    ("이영희", "younghee@example.com", "2025-01-15"),
    ("박민수", "minsu@example.com", "2025-02-01"),
    ("최지현", "jihyun@example.com", "2025-02-10"),
    ("정우성", "woosung@example.com", "2025-02-20"),
]

cur.executemany(
    "INSERT INTO users (name, email, join_date) VALUES (?, ?, ?)",
    users_data,
)

conn.commit()
print("👤 users 데이터 삽입 완료")


👤 users 데이터 삽입 완료


In [7]:
# 2-2) products 데이터 삽입
products_data = [
    ("무선 이어폰", "전자제품", 99000),
    ("블루투스 스피커", "전자제품", 59000),
    ("데이터베이스 입문", "도서", 22000),
    ("파이썬 프로그래밍", "도서", 28000),
    ("인공지능 개론", "도서", 32000),
    ("인체공학 의자", "생활용품", 199000),
    ("스탠드 조명", "생활용품", 39000),
    ("노트북 거치대", "생활용품", 29000),
]

cur.executemany(
    "INSERT INTO products (name, category, price) VALUES (?, ?, ?)",
    products_data,
)

conn.commit()
print("📦 products 데이터 삽입 완료")


📦 products 데이터 삽입 완료


In [8]:
# 2-3) orders 및 order_items, reviews 데이터 삽입

today = datetime(2025, 11, 27)

orders_data = [
    # (user_id, order_date, total_amount)
    (1, (today - timedelta(days=40)).strftime("%Y-%m-%d"), 99000),          # 김철수, 무선 이어폰
    (2, (today - timedelta(days=35)).strftime("%Y-%m-%d"), 81000),          # 이영희, 도서 2권
    (3, (today - timedelta(days=20)).strftime("%Y-%m-%d"), 199000),        # 박민수, 의자
    (1, (today - timedelta(days=10)).strftime("%Y-%m-%d"), 88000),         # 김철수, 스피커+조명
    (4, (today - timedelta(days=5)).strftime("%Y-%m-%d"), 61000),          # 최지현, 도서+거치대
    (5, (today - timedelta(days=2)).strftime("%Y-%m-%d"), 131000),         # 정우성, 무선이어폰+책
]

cur.executemany(
    "INSERT INTO orders (user_id, order_date, total_amount) VALUES (?, ?, ?)",
    orders_data,
)
conn.commit()

# 생성된 order_id 확인
cur.execute("SELECT id, user_id, order_date, total_amount FROM orders")
orders_list = cur.fetchall()
orders_list


[(1, 1, '2025-10-18', 99000.0),
 (2, 2, '2025-10-23', 81000.0),
 (3, 3, '2025-11-07', 199000.0),
 (4, 1, '2025-11-17', 88000.0),
 (5, 4, '2025-11-22', 61000.0),
 (6, 5, '2025-11-25', 131000.0)]

In [9]:
# order_items 삽입
# 각 주문별로 어떤 상품을 몇 개 샀는지 정의합니다.

order_items_data = [
    # order 1: 김철수, 무선 이어폰 1개
    (1, 1, 1, 99000),  # (order_id, product_id, quantity, price)

    # order 2: 이영희, 데이터베이스 입문 + 파이썬 프로그래밍
    (2, 3, 1, 22000),
    (2, 4, 1, 28000),

    # order 3: 박민수, 인체공학 의자
    (3, 6, 1, 199000),

    # order 4: 김철수, 스피커 + 조명
    (4, 2, 1, 59000),
    (4, 7, 1, 39000),

    # order 5: 최지현, 인공지능 개론 + 노트북 거치대
    (5, 5, 1, 32000),
    (5, 8, 1, 29000),

    # order 6: 정우성, 무선 이어폰 + 데이터베이스 입문
    (6, 1, 1, 99000),
    (6, 3, 1, 22000),
]

cur.executemany(
    "INSERT INTO order_items (order_id, product_id, quantity, price) VALUES (?, ?, ?, ?)",
    order_items_data,
)

conn.commit()
print("🧾 orders & order_items 데이터 삽입 완료")


🧾 orders & order_items 데이터 삽입 완료


In [10]:
# 2-4) reviews 데이터 삽입

reviews_data = [
    # (user_id, product_id, rating, comment, created_at)
    (1, 1, 5, "음질 좋고 배터리 오래갑니다.", (today - timedelta(days=38)).strftime("%Y-%m-%d")),
    (2, 3, 4, "데이터베이스 기초 공부에 도움이 됩니다.", (today - timedelta(days=34)).strftime("%Y-%m-%d")),
    (2, 4, 5, "예제가 많아서 실습하기 좋아요.", (today - timedelta(days=33)).strftime("%Y-%m-%d")),
    (3, 6, 4, "장시간 앉아 있어도 편합니다.", (today - timedelta(days=18)).strftime("%Y-%m-%d")),
    (1, 2, 4, "가격 대비 성능이 괜찮아요.", (today - timedelta(days=9)).strftime("%Y-%m-%d")),
    (4, 5, 5, "AI 입문용으로 강추합니다.", (today - timedelta(days=4)).strftime("%Y-%m-%d")),
    (5, 1, 4, "디자인 예쁘고 무난합니다.", (today - timedelta(days=1)).strftime("%Y-%m-%d")),
]

cur.executemany(
    "INSERT INTO reviews (user_id, product_id, rating, comment, created_at) VALUES (?, ?, ?, ?, ?)",
    reviews_data,
)

conn.commit()
print("⭐ reviews 데이터 삽입 완료")


⭐ reviews 데이터 삽입 완료


## 3. 간단 SQL 질의 예시

LLM을 붙이기 전에, 우리가 만든 데이터 구조를 이해하기 위해 **기본 SQL 질의**를 몇 개 실행해 봅니다.


In [11]:
def run_sql(query: str, params=None):
    """도움 함수: SQL을 실행하고 DataFrame으로 결과를 보여줍니다."""
    if params is None:
        params = []
    df = pd.read_sql_query(query, conn, params=params)
    display(df)

print("✅ run_sql 함수 준비 완료")


✅ run_sql 함수 준비 완료


In [12]:
# 예시 1: 모든 상품 조회
print("[모든 상품 목록]")
run_sql("SELECT * FROM products")

# 예시 2: 최근 30일 주문 내역
print("[최근 30일 주문]")
recent_date = (today - timedelta(days=30)).strftime("%Y-%m-%d")
run_sql(
    """
    SELECT o.id AS order_id, u.name AS user_name, o.order_date, o.total_amount
    FROM orders o
    JOIN users u ON o.user_id = u.id
    WHERE o.order_date >= ?
    ORDER BY o.order_date;
    """,
    [recent_date],
)


[모든 상품 목록]


,id,name,category,price
0,1,무선 이어폰,전자제품,99000.0
1,2,블루투스 스피커,전자제품,59000.0
2,3,데이터베이스 입문,도서,22000.0
3,4,파이썬 프로그래밍,도서,28000.0
4,5,인공지능 개론,도서,32000.0
5,6,인체공학 의자,생활용품,199000.0
6,7,스탠드 조명,생활용품,39000.0
7,8,노트북 거치대,생활용품,29000.0


[최근 30일 주문]


,order_id,user_name,order_date,total_amount
0,3,박민수,2025-11-07,199000.0
1,4,김철수,2025-11-17,88000.0
2,5,최지현,2025-11-22,61000.0
3,6,정우성,2025-11-25,131000.0


In [13]:
# 예시 3: 상품별 평균 평점
print("[상품별 평균 평점]")
run_sql(
    """
    SELECT p.id, p.name, AVG(r.rating) AS avg_rating, COUNT(r.id) AS review_count
    FROM products p
    LEFT JOIN reviews r ON p.id = r.product_id
    GROUP BY p.id, p.name
    ORDER BY avg_rating DESC;
    """
)


[상품별 평균 평점]


,id,name,avg_rating,review_count
0,4,파이썬 프로그래밍,5.0,1
1,5,인공지능 개론,5.0,1
2,1,무선 이어폰,4.5,2
3,2,블루투스 스피커,4.0,1
4,3,데이터베이스 입문,4.0,1
5,6,인체공학 의자,4.0,1
6,7,스탠드 조명,NaN,0
7,8,노트북 거치대,NaN,0


## 4. OpenAI LLM 설정

LLM을 사용하려면 **API 키**가 필요합니다.

1. OpenAI 계정에서 API 키를 발급받습니다.
2. Colab에서 아래와 같이 환경변수에 설정합니다.

⚠️ **주의:**
- API 키는 절대 노트북을 공유할 때 그대로 남겨두면 안 됩니다.
- 수업에서는 개인 계정이 아닌 별도의 실습용 키/환경을 사용하는 것이 안전합니다.


In [14]:
# 👇 이 셀에서 직접 키를 적지 말고, Colab 메뉴의
#   [실행] → [환경변수 관리] 기능을 활용하는 것을 추천합니다.
# 예시) os.environ["OPENAI_API_KEY"] = "sk-..."  # <-- 실제 키로 교체

from google.colab import userdata

# Colab Secret에서 API Key 로딩
raw_key = userdata.get("OPENAI_API_KEY")
api_key = raw_key.strip() if raw_key else None

if api_key is None:
    raise ValueError(
        "❌ ERROR: Colab userdata에 OPENAI_API_KEY가 없습니다.\n"
        "Colab 왼쪽 메뉴 → 🔑 Secret → OPENAI_API_KEY 추가 후 런타임을 재시작하세요."
    )

# OpenAI 클라이언트 생성
client = OpenAI(api_key=api_key)
print("✅ OpenAI API Key Loaded (prefix):", api_key[:8], "***")

✅ OpenAI API Key Loaded (prefix): sk-proj- ***


## 5. 자연어 → SQL 변환 함수 설계

### 아이디어
- LLM에게 **"너는 SQLite 조교"**라고 역할을 주고,
- 우리가 가진 테이블 구조를 **텍스트로 설명**해 준 뒤,
- 사용자의 한국어 질문을 보고 **SQL 쿼리와 한국어 설명**을 만들어 달라고 요청합니다.

### 중요한 제약 조건
- **SELECT 문만** 허용 (데이터를 변경하는 UPDATE/DELETE/INSERT 금지)
- **여러 문장을 한 번에 실행하는 쿼리 금지**
- **존재하지 않는 테이블/컬럼 이름 사용 금지**
- 결과는 반드시 JSON 형식으로 (`sql`, `explanation_ko`)


In [15]:
schema_description = textwrap.dedent(
    """
    [데이터베이스 스키마]

    1) users
      - id (INTEGER, PK)
      - name (TEXT)
      - email (TEXT)
      - join_date (TEXT, 'YYYY-MM-DD')

    2) products
      - id (INTEGER, PK)
      - name (TEXT)
      - category (TEXT)  -- 예: '전자제품', '도서', '생활용품'
      - price (REAL)

    3) orders
      - id (INTEGER, PK)
      - user_id (INTEGER, FK → users.id)
      - order_date (TEXT, 'YYYY-MM-DD')
      - total_amount (REAL)

    4) order_items
      - id (INTEGER, PK)
      - order_id (INTEGER, FK → orders.id)
      - product_id (INTEGER, FK → products.id)
      - quantity (INTEGER)
      - price (REAL)   -- 주문 시점의 가격

    5) reviews
      - id (INTEGER, PK)
      - user_id (INTEGER, FK → users.id)
      - product_id (INTEGER, FK → products.id)
      - rating (INTEGER, 1~5)
      - comment (TEXT)
      - created_at (TEXT, 'YYYY-MM-DD')
    """
)

print(schema_description)



[데이터베이스 스키마]

1) users
  - id (INTEGER, PK)
  - name (TEXT)
  - email (TEXT)
  - join_date (TEXT, 'YYYY-MM-DD')

2) products
  - id (INTEGER, PK)
  - name (TEXT)
  - category (TEXT)  -- 예: '전자제품', '도서', '생활용품'
  - price (REAL)

3) orders
  - id (INTEGER, PK)
  - user_id (INTEGER, FK → users.id)
  - order_date (TEXT, 'YYYY-MM-DD')
  - total_amount (REAL)

4) order_items
  - id (INTEGER, PK)
  - order_id (INTEGER, FK → orders.id)
  - product_id (INTEGER, FK → products.id)
  - quantity (INTEGER)
  - price (REAL)   -- 주문 시점의 가격

5) reviews
  - id (INTEGER, PK)
  - user_id (INTEGER, FK → users.id)
  - product_id (INTEGER, FK → products.id)
  - rating (INTEGER, 1~5)
  - comment (TEXT)
  - created_at (TEXT, 'YYYY-MM-DD')



In [16]:
def nl_to_sql(question: str) -> dict:
    """자연어 질문을 입력받아 SQL과 한국어 설명을 JSON으로 반환합니다.

    반환 형식 예시:
    {
      "sql": "SELECT ...;",
      "explanation_ko": "이 쿼리는 ..."
    }
    """
    if client is None:
        raise RuntimeError("OpenAI client가 설정되지 않았습니다. API 키를 확인하세요.")

    system_prompt = f"""
당신은 SQLite 데이터베이스 조교입니다.
아래에 주어진 스키마 정보만을 사용하여 SQL SELECT 문을 작성하세요.

{schema_description}

[규칙]
1. 항상 하나의 SELECT 문만 작성합니다. 여러 문장을 세미콜론으로 이어서 쓰지 마세요.
2. INSERT, UPDATE, DELETE 등 데이터 변경 구문은 절대 사용하지 마세요. (읽기 전용)
3. 존재하지 않는 테이블이나 컬럼 이름을 지어내지 말고, 위에 정의된 것만 사용하세요.
4. GROUP BY, ORDER BY, LIMIT, JOIN 등을 자유롭게 활용할 수 있습니다.
5. 결과를 JSON 형식의 문자열로만 출력하세요. 파이썬 코드나 다른 설명은 포함하지 마세요.

[JSON 출력 형식]
{{
  "sql": "SELECT ...",
  "explanation_ko": "이 쿼리는 ...을 조회합니다. 결과는 ...을 의미합니다."
}}
"""

    user_prompt = f"[질문]\n{question}"

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.1,
    )

    raw = response.choices[0].message.content
    print("[LLM 원본 응답]\n", raw)

    try:
        data = json.loads(raw)
    except Exception as e:
        print("JSON 파싱 실패:", e)
        data = {"sql": None, "explanation_ko": "JSON 파싱에 실패했습니다."}
    return data

print("✅ nl_to_sql 함수 정의 완료")


✅ nl_to_sql 함수 정의 완료


In [17]:
def run_nl_query(question: str):
    """자연어 질문을 받아 SQL로 변환하고 실행 결과까지 보여주는 편의 함수"""
    print("\n============================")
    print("[사용자 질문]", question)
    print("============================")

    data = nl_to_sql(question)
    sql = data.get("sql")
    explanation = data.get("explanation_ko")

    if not sql:
        print("⚠️ SQL이 생성되지 않았습니다.")
        print("설명:", explanation)
        return

    print("\n[생성된 SQL]\n", sql)

    try:
        df = pd.read_sql_query(sql, conn)
    except Exception as e:
        print("❌ SQL 실행 중 오류 발생:", e)
        return

    print("\n[쿼리 결과]")
    display(df)

    print("\n[설명]")
    print(explanation)

print("✅ run_nl_query 함수 정의 완료")


✅ run_nl_query 함수 정의 완료


## 6. 자연어로 쇼핑·리뷰 데이터 질의해 보기

아래 예시 질문들을 실행해 보면서 LLM이 어떤 SQL을 생성하는지, 그리고 결과가 잘 나오는지 확인합니다.


In [18]:
# ⚠️ 이 셀을 실행하기 전에, 반드시 OPENAI_API_KEY를 설정해야 합니다.

example_questions = [
    "최근 30일 동안 주문이 가장 많은 상품 3개를 알려줘.",
    "평균 평점이 가장 높은 상품 5개와 그 평점을 보여줘.",
    "김철수가 구매한 상품 목록과 주문 날짜를 보고 싶어.",
    "도서(category가 '도서') 중에서 평균 별점이 4점 이상인 상품을 알려줘.",
    "사용자별로 총 주문 금액을 계산해서, 많이 쓴 순서대로 정렬해줘.",
]

for q in example_questions:
    run_nl_query(q)



[사용자 질문] 최근 30일 동안 주문이 가장 많은 상품 3개를 알려줘.
[LLM 원본 응답]
 {
  "sql": "SELECT json_group_array(json_object('product_id', p.id, 'product_name', p.name, 'total_quantity', total_qty)) AS result_json FROM (SELECT oi.product_id, p.name, SUM(oi.quantity) AS total_qty FROM order_items oi JOIN orders o ON oi.order_id = o.id JOIN products p ON oi.product_id = p.id WHERE o.order_date >= date('now', '-30 days') GROUP BY oi.product_id ORDER BY total_qty DESC LIMIT 3) AS top_products;",
  "explanation_ko": "이 쿼리는 최근 30일 동안 주문된 상품들 중에서 주문 수량이 가장 많은 상위 3개 상품의 ID, 이름, 총 주문 수량을 조회합니다. 결과는 JSON 배열 형태로 반환되어 각 상품의 정보를 쉽게 확인할 수 있습니다."
}

[생성된 SQL]
 SELECT json_group_array(json_object('product_id', p.id, 'product_name', p.name, 'total_quantity', total_qty)) AS result_json FROM (SELECT oi.product_id, p.name, SUM(oi.quantity) AS total_qty FROM order_items oi JOIN orders o ON oi.order_id = o.id JOIN products p ON oi.product_id = p.id WHERE o.order_date >= date('now', '-30 days') GROUP BY oi.product_id ORDER BY tot

,result_json
0,"[{""product_name"":""무선 이어폰"",""order_date"":""2025-1..."



[설명]
이 쿼리는 이름이 '김철수'인 사용자가 구매한 상품 목록과 각 상품의 주문 날짜를 조회합니다. 결과는 각 상품명과 주문 날짜를 포함하는 JSON 배열 형태로 반환됩니다.

[사용자 질문] 도서(category가 '도서') 중에서 평균 별점이 4점 이상인 상품을 알려줘.
[LLM 원본 응답]
 {
  "sql": "SELECT json_group_array(json_object('product_id', p.id, 'product_name', p.name, 'average_rating', avg(r.rating))) AS result FROM products p JOIN reviews r ON p.id = r.product_id WHERE p.category = '도서' GROUP BY p.id HAVING avg(r.rating) >= 4",
  "explanation_ko": "이 쿼리는 '도서' 카테고리에 속하는 상품들 중에서 리뷰의 평균 별점이 4점 이상인 상품들의 ID, 이름, 평균 별점을 JSON 배열 형태로 조회합니다."
}

[생성된 SQL]
 SELECT json_group_array(json_object('product_id', p.id, 'product_name', p.name, 'average_rating', avg(r.rating))) AS result FROM products p JOIN reviews r ON p.id = r.product_id WHERE p.category = '도서' GROUP BY p.id HAVING avg(r.rating) >= 4
❌ SQL 실행 중 오류 발생: Execution failed on sql 'SELECT json_group_array(json_object('product_id', p.id, 'product_name', p.name, 'average_rating', avg(r.rating))) AS result FROM products p JOIN reviews r ON p.id = r.

,user_id,name,total_spent
0,3,박민수,199000.0
1,1,김철수,187000.0
2,5,정우성,131000.0
3,2,이영희,81000.0
4,4,최지현,61000.0



[설명]
이 쿼리는 각 사용자의 총 주문 금액을 계산하여 많이 쓴 순서대로 정렬합니다. 사용자가 주문을 하지 않았더라도 이름과 함께 총 주문 금액 0으로 표시됩니다.


## 7. (선택) 매출 · 리뷰 리포트 자동 생성

이번에는 LLM을 **자연어 질의 변환** 뿐만 아니라,
**데이터를 요약해서 리포트를 작성하는 데에도** 사용해 보겠습니다.

### 아이디어
1. Python + SQL로 집계 데이터(예: 최근 30일 매출, 인기 상품, 평균 평점)를 먼저 계산.
2. 그 결과를 LLM에 전달하고, **사람이 읽기 좋은 요약 리포트**를 요청.


In [19]:
def get_sales_summary(days: int = 30) -> dict:
    """최근 N일간 매출/주문/상품/리뷰 통계를 집계합니다."""
    since = (today - timedelta(days=days)).strftime("%Y-%m-%d")

    # 1) 총 매출, 주문 수
    total_df = pd.read_sql_query(
        """
        SELECT
            COUNT(*) AS num_orders,
            SUM(total_amount) AS total_revenue,
            AVG(total_amount) AS avg_order_amount
        FROM orders
        WHERE order_date >= ?;
        """,
        conn,
        params=[since],
    )

    # 2) 상품별 판매량 Top 5
    top_products_df = pd.read_sql_query(
        """
        SELECT p.id, p.name, p.category,
               SUM(oi.quantity) AS total_qty,
               SUM(oi.price * oi.quantity) AS product_revenue
        FROM order_items oi
        JOIN orders o ON oi.order_id = o.id
        JOIN products p ON oi.product_id = p.id
        WHERE o.order_date >= ?
        GROUP BY p.id, p.name, p.category
        ORDER BY total_qty DESC
        LIMIT 5;
        """,
        conn,
        params=[since],
    )

    # 3) 상품별 평균 평점
    rating_df = pd.read_sql_query(
        """
        SELECT p.id, p.name,
               AVG(r.rating) AS avg_rating,
               COUNT(r.id) AS review_count
        FROM products p
        LEFT JOIN reviews r ON p.id = r.product_id
        GROUP BY p.id, p.name
        ORDER BY avg_rating DESC;
        """,
        conn,
    )

    return {
        "since": since,
        "total": total_df.to_dict(orient="records"),
        "top_products": top_products_df.to_dict(orient="records"),
        "ratings": rating_df.to_dict(orient="records"),
    }

summary_example = get_sales_summary(30)
print(json.dumps(summary_example, ensure_ascii=False, indent=2))


{
  "since": "2025-10-28",
  "total": [
    {
      "num_orders": 4,
      "total_revenue": 479000.0,
      "avg_order_amount": 119750.0
    }
  ],
  "top_products": [
    {
      "id": 1,
      "name": "무선 이어폰",
      "category": "전자제품",
      "total_qty": 1,
      "product_revenue": 99000.0
    },
    {
      "id": 2,
      "name": "블루투스 스피커",
      "category": "전자제품",
      "total_qty": 1,
      "product_revenue": 59000.0
    },
    {
      "id": 3,
      "name": "데이터베이스 입문",
      "category": "도서",
      "total_qty": 1,
      "product_revenue": 22000.0
    },
    {
      "id": 5,
      "name": "인공지능 개론",
      "category": "도서",
      "total_qty": 1,
      "product_revenue": 32000.0
    },
    {
      "id": 6,
      "name": "인체공학 의자",
      "category": "생활용품",
      "total_qty": 1,
      "product_revenue": 199000.0
    }
  ],
  "ratings": [
    {
      "id": 4,
      "name": "파이썬 프로그래밍",
      "avg_rating": 5.0,
      "review_count": 1
    },
    {
      "id": 5,
      "name": "인공지능

In [20]:
def generate_sales_report(days: int = 30):
    """최근 N일간의 매출/리뷰 현황에 대한 자연어 리포트를 생성합니다."""
    if client is None:
        raise RuntimeError("OpenAI client가 설정되지 않았습니다. API 키를 확인하세요.")

    summary = get_sales_summary(days)
    summary_json = json.dumps(summary, ensure_ascii=False)

    system_prompt = """
당신은 데이터 분석 리포트를 작성하는 조교입니다.
주어진 JSON 데이터를 바탕으로, 한국어로 간단한 매출/리뷰 요약 리포트를 작성하세요.

[요구사항]
1. 전체 매출 규모와 주문 수에 대해 한 단락으로 설명합니다.
2. 판매량이 높은 주요 상품(Top 3)을 언급하고, 특징을 간단히 서술합니다.
3. 평균 평점이 높은 상품이 있다면 몇 개만 예로 들어 소개합니다.
4. 전체 길이는 A4 반쪽 정도 분량(5~10문장)으로 작성합니다.
"""

    user_prompt = f"최근 {days}일간의 매출 및 리뷰 통계 JSON은 다음과 같습니다.\n\n{summary_json}"

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.4,
    )

    report = response.choices[0].message.content
    print(report)

print("✅ generate_sales_report 함수 정의 완료")


✅ generate_sales_report 함수 정의 완료


In [ ]:
# 예시: 최근 30일간 쇼핑몰 요약 리포트 생성
generate_sales_report(30)


최근 30일간 총 4건의 주문이 이루어져 총 매출은 479,000원으로 집계되었습니다. 주문당 평균 구매 금액은 약 119,750원으로 비교적 높은 편입니다. 판매량 기준 상위 3개 상품은 각각 무선 이어폰, 블루투스 스피커, 데이터베이스 입문 도서로, 전자제품과 도서 카테고리에 속해 있습니다. 특히 무선 이어폰은 99,000원의 매출을 기록하며 전자제품 중 가장 높은 매출을 올렸고, 데이터베이스 입문은 입문용 도서로서 꾸준한 수요가 있음을 보여줍니다. 평균 평점이 높은 상품으로는 파이썬 프로그래밍과 인공지능 개론 도서가 각각 5.0점 만점에 평점을 받았으며, 무선 이어폰도 4.5점으로 좋은 평가를 받았습니다. 전반적으로 전자제품과 IT 관련 도서가 매출과 리뷰에서 긍정적인 반응을 얻고 있어 향후 이 분야의 상품 구성이 중요할 것으로 판단됩니다.


## 마무리

이 노트북에서 구현한 내용은 학부 기말 프로젝트로 사용하기에 적당한 기본 골격입니다.

- **DB 설계**: 5개 테이블, PK/FK, 간단한 제약조건
- **SQL 활용**: JOIN, GROUP BY, 집계, 날짜 필터링
- **LLM 연동**:
  - 자연어 → SQL 변환
  - 결과 해석 및 설명
  - 매출/리뷰 요약 리포트 자동 생성

학생들은 이 구조를 바탕으로:
- 더 다양한 질문 예시를 추가하거나,
- 추천 기능(예: "당신에게 맞는 상품 추천")을 확장하거나,
- 사용자 인터페이스(예: Streamlit, Gradio 등)를 붙여서 웹 서비스로 발전시킬 수 있습니다.
